<font size=8> Network Tests


In [1]:
#Common Python Packages
from pickle import TRUE
import pickle
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#dynapse2 spikegen imports
from lib.dynapse2_util import *
from lib.dynapse2_network import Network
from lib.dynapse2_spikegen import send_events,get_fpga_time, send_virtual_events, poisson_gen, isi_gen,regular_gen
from lib.dynapse2_raster import *
from lib.dynapse2_obj import *
from lib.dynapse2_raster import *

#Statistics imports
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from itertools import product

#my imports
from adaptation import pc_single,ff,pc_pv_sst,ff_network
from adaptation_lib.spike_stats import *
from adaptation_lib.dynapse_setup import *
from adaptation_lib.graphing import *
from configs import neuron_configs_bio

# Determine the user's home directory
home_directory = os.path.expanduser("~")
# Path to the Documents directory
documents_path = os.path.join(home_directory, "Documents")
# Path to the dynapse-se2-data directory within Documents
save_directory_path = os.path.join(documents_path, "dynapse-se2-data")
# Path to the data
data_type_path = os.path.join(save_directory_path, "network_heatmaps")

In [2]:
[board,profile_path,number_of_chips]=obtain_board()

<font size=8> SWEEP MAIN CONFIG

In [181]:
neuron_config=neuron_configs_bio.config()
#remove SST connections to focus in th eEI balance structure

#input frequencies and conneciton ratiors
neuron_config['PV_SST']=0
neuron_config['SST_PC']=0
neuron_config['SST_PV']=0
neuron_config['Input_SST']=0
neuron_config['PC_SST']=0
neuron_config['PC_Adaptation']=True

nvn=15
pvn=20
pcn=100
sstn=0
bin_size=0.060
duration=1

<font size=6> Probe

In [ ]:
neuron_config['Input_Frequency']=1
neuron_config['PC_PC']=.1
neuron_config['PC_PV']=.1
neuron_config['input_type']='Poisson'

[output_events,test_config]=pc_pv_sst.pc_pv_sst(board=board, profile_path=profile_path, number_of_chips=number_of_chips,neuron_config=neuron_config)
raster_data=population_data(output_events, nvn, pcn, pvn, sstn)
rates = spike_count(output_events=output_events)
PC_rates, _, _ = pop_rates(rates, test_config)
cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)


plot_network_raster_psth(raster_data,duration, bin_size)

In [10]:
def run_sweep(sweep_range_1, sweep_range_2, config_var_name_1, config_var_name_2, sweep_name, save_name, data_type_path, neuron_config):
    #create the dapa path with the test name
    data_path=os.path.join(data_type_path, f"{sweep_name}.npy")

    #obtain previous data if it exists if not create a new dictionary with the data
    if os.path.exists(data_path):
        simulation_data = np.load(data_path, allow_pickle=True).item()
    else:
        simulation_data = {
            "cv_values_pc": [np.nan] * len(sweep_range_1) * len(sweep_range_2),
            "cv_values_pv": [np.nan] * len(sweep_range_1) * len(sweep_range_2),
            "synchrony_values_pc": [np.nan] * len(sweep_range_1) * len(sweep_range_2),
            "synchrony_values_pv": [np.nan] * len(sweep_range_1) * len(sweep_range_2),
            "mean_pc_rates": [np.nan] * len(sweep_range_1) * len(sweep_range_2),
            config_var_name_1: sweep_range_1,
            config_var_name_2: sweep_range_2
        }
    
    simulation_data['input_frequency'] =sweep_range_1
    simulation_data['connection_ratios'] =sweep_range_2   


    #run the test for each value in the sweep range
    for value_1 in simulation_data[config_var_name_1]:
        neuron_config[config_var_name_1] = np.round(value_1,2)
        for value_2 in simulation_data[config_var_name_2]:
            if contains_nan(simulation_data,config_var_name_1,config_var_name_2,value_1, value_2):
                neuron_config[config_var_name_2] = np.round(value_2,2)
                try:
                    [output_events, test_config] = run_simulation(neuron_config)

                    cv_temp, synchrony_temp = run_dynamic_anal(output_events, test_config)

                    rates = spike_count(output_events=output_events)
                    PC_rates, _, _ = pop_rates(rates, test_config)

                    index = len(simulation_data[config_var_name_2]) * list(simulation_data[config_var_name_1]).index(value_1) + list(simulation_data[config_var_name_2]).index(value_2)
                    simulation_data["cv_values_pc"][index] = cv_temp[0]
                    simulation_data["cv_values_pv"][index] = cv_temp[1]
                    simulation_data["synchrony_values_pc"][index] = synchrony_temp[0]
                    simulation_data["synchrony_values_pv"][index] = synchrony_temp[1]
                    simulation_data["mean_pc_rates"][index] = np.mean(PC_rates)
                except timeout_decorator.TimeoutError:
                    print(f"Simulation timed out for {config_var_name_1} {value_1} and {config_var_name_2} {value_2}")
                    index = len(simulation_data[config_var_name_2]) * list(simulation_data[config_var_name_1]).index(value_1) + list(simulation_data[config_var_name_2]).index(value_2)
                    simulation_data["cv_values_pc"][index] = np.nan
                    simulation_data["cv_values_pv"][index] = np.nan
                    simulation_data["synchrony_values_pc"][index] = np.nan
                    simulation_data["synchrony_values_pv"][index] = np.nan
                    simulation_data["mean_pc_rates"][index] = np.nan
                    
                np.save(os.path.join(data_type_path, f"{save_name}.npy"), simulation_data)


    return simulation_data

@timeout_decorator.timeout(40)# Set a 5-second timeout
def run_simulation(neuron_config):
    return pc_pv_sst.pc_pv_sst(
        board=board,
        profile_path=profile_path,
        number_of_chips=number_of_chips,
        neuron_config=neuron_config
    )

<font size=6> Poisson PC PC


In [ ]:
sweep_range_1=np.arange(1,31,1)
sweep_range_2=np.arange(0,.7,.1)

config_var_name_1='input_frequency'
neuron_config['input_type']='Poisson'
config_var_name_2='PC_PC'
sweep_name='EI_sweep_PC_PC_poisson'

run_sweep(sweep_range_1, sweep_range_2, config_var_name_1, config_var_name_2, sweep_name,sweep_name, data_type_path, neuron_config)

<font size=6> Poisson PC PV


In [ ]:

neuron_config['input_type']='Poisson'

#input frequencies and conneciton ratiors
sweep_range_1=np.arange(1,31,1)
sweep_range_2=np.arange(0,.7,.1)
config_var_name_1='input_frequency'
config_var_name_2='PC_PV'
sweep_name='EI_sweep_PC_PV_poisson'

run_sweep(sweep_range_1, sweep_range_2, config_var_name_1, config_var_name_2, sweep_name,sweep_name, data_type_path, neuron_config)

<font size=6> Regular PC PC


In [ ]:

neuron_config['input_type']='Regular'

#input frequencies and conneciton ratiors
sweep_range_1=np.arange(1,31,1)
sweep_range_2=np.arange(0,.7,.1)
config_var_name_1='input_frequency'
config_var_name_2='PC_PC'
sweep_name='EI_sweep_PC_PC_regular'

run_sweep(sweep_range_1, sweep_range_2, config_var_name_1, config_var_name_2, sweep_name,sweep_name, data_type_path, neuron_config)

<font size=6> Regular PC PV


In [2]:

neuron_config['input_type']='Regular'

#input frequencies and conneciton ratiors
sweep_range_1=np.arange(1,31,1)
sweep_range_2=np.arange(0,.7,.1)
config_var_name_1='input_frequency'
config_var_name_2='PC_PV'
sweep_name='EI_sweep_PC_PV_regular'

run_sweep(sweep_range_1, sweep_range_2, config_var_name_1, config_var_name_2, sweep_name,sweep_name, data_type_path, neuron_config)

NameError: name 'neuron_config' is not defined